In [99]:
#Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split as tt_split 
from sklearn.preprocessing import StandardScaler as SScaler

from sklearn.linear_model import LinearRegression as Lreg
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.metrics import mean_squared_log_error as MSLE #se usa este indicador porque es lo que se pide optimizar

In [32]:
#cargando train set
df = pd.read_csv('train.csv')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [54]:
#Diferencia entre set de entrenamiento y prueba
#Esta parte es necesaria dada la competencia, no seria un tema si fuera un caso real producto 
#de que nos encargariamos de que la distribucion tuviera las mismas caracteristicas en 
#entrenamiento y prueba

In [37]:
train_info = df.describe().iloc[1,:]

In [38]:
df_test = pd.read_csv('test.csv')

In [39]:
test_info = df_test.describe().iloc[1,:]

In [55]:
#Diferencia Absoluta
abs((train_info - test_info)/test_info).sort_values(ascending=False)

3SsnPorch        0.900149
Id               0.666438
LowQualFinSF     0.649353
PoolArea         0.581627
MiscVal          0.252354
BsmtHalfBath     0.117606
ScreenPorch      0.117406
BsmtFinSF2       0.115357
EnclosedPorch    0.094426
LotArea          0.071052
2ndFlrSF         0.064499
Fireplaces       0.054702
MoSold           0.035670
OpenPorchSF      0.034227
MasVnrArea       0.029552
BsmtUnfSF        0.023355
LotFrontage      0.021429
BsmtFullBath     0.020973
TotRmsAbvGrd     0.020769
GrLivArea        0.019796
HalfBath         0.013824
WoodDeckSF       0.011481
TotalBsmtSF      0.010813
BsmtFinSF1       0.010100
MSSubClass       0.008384
1stFlrSF         0.005268
BedroomAbvGr     0.004355
KitchenAbvGr     0.003914
OverallCond      0.003878
FullBath         0.003737
OverallQual      0.003371
YearRemodAdd     0.000606
GarageCars       0.000569
GarageArea       0.000447
GarageYrBlt      0.000397
YearBuilt        0.000046
YrSold           0.000023
SalePrice             NaN
Name: mean, 

In [85]:
#Se puede ver

In [ ]:
#Explorando las variables

In [64]:
from pandas_profiling import ProfileReport as PR

In [67]:
prof = PR(df)
prof.to_file(output_file='train_ds.html')

prof2 = PR(df_test)
prof.to_file(output_file='test_ds.html')

C:\Users\tcorn\AppData\Roaming\Python\Python37\site-packages\pandas_profiling\model\correlations.py:107: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [ ]:
#independientes listas
df[['OverallQual','OverallCond','YearBuilt',\
 'TotalBsmtSF','GarageYrBlt','Fireplaces',\
 'GarageArea','FullBath','HalfBath',\
 '2ndFlrSF','GrLivArea','YearRemodAdd',\
 'MasVnrArea'
 ]].values

In [ ]:
#Funcion de Preproceso

In [247]:
def Preprocessing(df,train_flg):
    
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df['GarageArea'] = df['GarageArea'].fillna(0)
    
        
    Scale = SScaler()

    if train_flg == 1:
        y = df['SalePrice'].values
        X = df[['OverallQual','OverallCond','YearBuilt',\
                 'TotalBsmtSF','GarageYrBlt','Fireplaces',\
                 'GarageArea','FullBath','HalfBath',\
                 '2ndFlrSF','GrLivArea','YearRemodAdd',\
                 'MasVnrArea'
                 ]].values


        X_train, X_test, y_train, y_test = tt_split(X, y, train_size= 0.67, random_state=42)



        X_train = Scale.fit_transform(X_train)

        X_test = Scale.fit_transform(X_test)

        X_train = np.nan_to_num(X_train, nan = 0)
        X_test = np.nan_to_num(X_test, nan = 0)
        
        return  X_train, X_test, y_train, y_test
    else:
        X = df[['OverallQual','OverallCond','YearBuilt',\
                 'TotalBsmtSF','GarageYrBlt','Fireplaces',\
                 'GarageArea','FullBath','HalfBath',\
                 '2ndFlrSF','GrLivArea','YearRemodAdd',\
                 'MasVnrArea'
                 ]].values
        
        X_val = Scale.fit_transform(X)
        Id = df['Id'].values
        
        X_val = np.nan_to_num(X_val, nan = 0)
        
        return Id , X_val

In [203]:
#Funcion que prueba Modelos

In [204]:
def Prueba_modelo(X_train, X_test, y_train, y_test , modelo_input, nombre_modelo):
    modelo = modelo_input 
    modelo.fit(X_train, y_train)
    error = np.sqrt(MSLE(y_test, modelo.predict(X_test)))
    nombre = nombre_modelo
    modelo_entrenado = modelo
    return [nombre, error, modelo]

In [205]:
#Modelos
Modelos = []

In [206]:
name = 'Linear'
model = Lreg()
Modelos.append([name,model])

In [207]:
name = 'Tree'
model = DTR()
Modelos.append([name,model])

In [208]:
name = 'RandomForest'
model = RFR()
Modelos.append([name,model])

In [209]:
name = 'MultiLayer'
model = MLPC()
Modelos.append([name,model])

In [210]:
Modelos

[['Linear',
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)],
 ['Tree',
  DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best')],
 ['RandomForest',
  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0,
                        min_impurity_split=None, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        n_estimators=100, n_jobs=None, oob_score=False,
                      

In [211]:
X_train, X_test, y_train, y_test = Preprocessing(df,1)

In [212]:
Output = []

In [213]:
for modelo in Modelos:
    Output.append(Prueba_modelo(X_train, X_test, y_train, y_test, modelo[1],modelo[0]))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [214]:
#Scaled
Output[:]

[['Linear',
  0.18197202080625474,
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)],
 ['Tree',
  0.20931002604080062,
  DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best')],
 ['RandomForest',
  0.1575578188259158,
  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0,
                        min_impurity_split=None, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        n_

In [192]:
#Funcion de Creacion de Resultado

In [248]:
Id, X_val = Preprocessing(df_test,0)

In [249]:
modelo = Output[2][2]
Upload = modelo.predict(X_val)

In [254]:
Resultado = pd.DataFrame(list(zip(Id,Upload)), columns = ['Id','SalePrice'])

In [256]:
Resultado.to_csv('submission.csv', index=False)